<h1> Imports and prep

In [ ]:
#airline data from here: https://www.kaggle.com/code/prashant111/tutorial-time-series-forecasting-with-prophet/data

In [ ]:
import ipydeps
ipydeps.pip(['plotly', 'prophet', 'numpy', 'pandas', 'matplotlib'])
from prophet import Prophet
from prophet.plot import plot_plotly
import plotly.offline as py
py.init_notebook_mode()
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.metrics import mean_absolute_error
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

<h1> Airline dataset analysis (from kaggle)

In [ ]:
#read, parse, prep, show original data
file = '/kaggle/input/air-passengers/AirPassengers.csv'
df = pd.read_csv(file)
df['Month'] = pd.DatetimeIndex(df['Month'])
df = df.rename(columns={'Month': 'ds',
                        '#Passengers': 'y'})
ax = df.set_index('ds').plot(figsize=(12, 8))
ax.set_ylabel('Monthly Number of Airline Passengers')
ax.set_xlabel('Date')
plt.show() 

In [ ]:
my_model = Prophet(interval_width=0.95) 
my_model.fit(df) #prepare model with your data
future_dates = my_model.make_future_dataframe(periods=36, freq='MS') #make df to put predictions in...
#set # of periods and freq
forecast = my_model.predict(future_dates) #make predictions
my_model.plot(forecast, uncertainty=True) #plot prediction; may show twice in notebook?

In [ ]:
fig1 = my_model.plot_components(forecast) #show components of prediction

In [ ]:
#find and plot changepoints
from prophet.plot import add_changepoints_to_plot
fig = my_model.plot(forecast) 
a = add_changepoints_to_plot(fig.gca(), my_model, forecast)

<h1> Car sales dataset analysis (from machine learning masters)</h1>
note this one grabs the data automatically

In [ ]:
#get data, prep, make model
df = pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-car-sales.csv', header=0)
df.columns = ['ds', 'y']
df['ds']= pd.to_datetime(df['ds'])
model = Prophet()
model.fit(df.drop(df.index[-12:])) #don't train on the last 12 months, so we can predict this period to compare...
#... with ground truth

In [ ]:
#prep df to hold preds + make the preds and plot
future = list()
for i in range(1, 13):
    date = '1968-%02d' % i
    future.append([date])
future = pd.DataFrame(future)
future.columns = ['ds']
future['ds'] = pd.to_datetime(future['ds'])
forecast = model.predict(future)
model.plot(forecast, uncertainty=True)

In [ ]:
# forecast the last 12 months, and compare with ground truth
forecast = model.predict(future)
y_true = df['y'][-12:].values
y_pred = forecast['yhat'].values
mae = mean_absolute_error(y_true, y_pred)
print('MAE: %.3f' % mae)
# plot expected vs actual
plt.plot(y_true, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()